#### Copyright 2022 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# How to set managed configurations for an app and retrieve app feedback

This notebook shows you how to support the [managed configurations](https://developers.google.com/android/management/managed-configurations-iframe) and how to [retrieve feedback from apps](https://developers.google.com/android/management/app-feedback) with the Android Management API. If an app supports managed configurations, sending keyed app states is recommended as a way to update IT admins on the status of the configurations they set. 

To check if an app supports managed configurations:

1. Go to [managed Google Play](https://play.google.com/work).
2. Search for, and click the app you’d like to check.
3. If the production version of the app supports managed configuration, the following message is shown under the Approve or Buy button: This app offers managed configuration.

## Setup

You can use the [Android Management API quickstart guide](https://developers.google.com/android/management/quickstart) to become familiar with how to use this colab notebook. The quickstart will help you create a cloud project, enroll an enterprise, and provision a device.

In this colab you will need to have:

*   Enrolled an enterprise.
*   Created a policy.
*   Provisioned an Android 6.0+ device under your enterprise.

You will use the resources (`Enterprises`, `Devices`, `Policies`) that you have created from the quickstart.


# Set up the managed configurations iframe


## Authenticating
To create and access resources, you need to authenticate with an account that has edit rights over your project. To start the authentication flow, **run the cell below**.

When you build a server-based solution, you should create a
[service account](https://developers.google.com/android/management/service-account)
so you don't need to authorize the access every time.

In [ ]:
import json
from apiclient.discovery import build
from google_auth_oauthlib.flow import Flow
import pprint

# To improve the readability of deeply nested data structures, we create a 
# helper function to pretty print a result:
prettyprint = pprint.PrettyPrinter(indent=4).pprint

# This is a public OAuth config that you can use to run this guide.
# However, use different credentials when building your own solution.
CLIENT_CONFIG = {
  'web': {
    'client_id':'882252295571-m8qpit609pj2hsql4j5k0bue1injrtd2.apps.googleusercontent.com',
    'auth_uri':'https://accounts.google.com/o/oauth2/auth',
    'token_uri':'https://oauth2.googleapis.com/token',
    'auth_provider_x509_cert_url':'https://www.googleapis.com/oauth2/v1/certs',
    'client_secret':'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'
  }
}
SCOPES = ['https://www.googleapis.com/auth/androidmanagement']
CALLBACK_URL = 'https://google.github.io/android-management-api-samples/oauth_callback.html'

# Run the OAuth flow.
flow = Flow.from_client_config(CLIENT_CONFIG, SCOPES)
flow.redirect_uri = CALLBACK_URL
auth_url, _ = flow.authorization_url()
print('Please visit this URL to authorize this application: {}'.format(auth_url))

code = input('Enter the authorization code: ')
flow.fetch_token(code=code)

# Create the API client.
androidmanagement = build('androidmanagement', 'v1', credentials=flow.credentials)

print('\nAuthentication succeeded.')

## Declare your enterprise

If you already have an enterprise, you can enter the enterprise name in the cell below and run the cell. If you do not have a enterprise, please go through the [general Quickstart](https://developers.google.com/android/management/quickstart)

In [ ]:
# Paste your enterprise name here.
# It should be of the format 'enterprises/LC...'
enterprise_name = ''

## Select an application to configure

You can check in [Google Play](https://play.google.com/work) if an application supports managed configuration.

In [ ]:
# Paste the package name of an application here.
package_name = ''

# There are two ways you can support managed configurations in your EMM console:

*   Add the managed configurations iframe to your console and apply settings via managedConfigurationTemplate in ApplicationPolicy
*   Create your own UI and apply the settings via managedConfiguration in ApplicationPolicy.

Managed configurations allow the organization's IT admin to remotely specify settings for apps. To view sample apps with a managed configuration, see [AppRestrictionSchema](https://github.com/android/enterprise-samples/tree/master/AppRestrictionSchema/#readme).

## Option 1: Set managed configuration using the managed configuration iframe

This section shows you how to support [managed configurations iframe](https://developers.google.com/android/management/managed-configurations-iframe) with the Android Management API. The managed configurations iframe is an embeddable UI that lets IT admins save, edit, and delete an app’s managed configuration settings.

### Render the managed configuration iframe

The iframe retrieves and displays the managed configurations schema for a specified app. Within the iframe, an IT admin can set configurations and save them as a configuration profile.

In [ ]:
IFRAME_URL = 'https://storage.googleapis.com/android-management-api-samples/managed_configuration_iframe.html'

# Creates a web token to access an embeddable managed Google Play web UI for a given enterprise.
web_token = androidmanagement.enterprises().webTokens().create(
    parent=enterprise_name,
    body={
      'parentFrameUrl': IFRAME_URL
    }
).execute()

full_iframe_url = IFRAME_URL + "?token=" + web_token["value"] + "&packageName=" + package_name

print('Open the managed configuration iframe:', full_iframe_url)

In [ ]:
# Each time an IT admin saves a new configuration profile, the iframe returns a unique identifier called mcmId.
# Paste the mcmId of the managed configurations profile you just created here.
mdm_id = ""

### Set the managed configuration profile in the policy

Policies (also called a policy) are the core resource of the Android Management API. You use them to create and save groups of device and app management settings for your customers to apply to devices. Each configuration profile is saved as a unique mcmId. To apply a configuration profile to a policy, specify mcmId in managedConfigurationTemplate.

In [ ]:
policy_name = enterprise_name + '/policies/managed-configurations'

policy = {
    'applications': [
        {
            'installType': 'FORCE_INSTALLED',
            'packageName': package_name,
            'managedConfigurationTemplate': {
                'templateId': mdm_id
            }
        }
    ],
    'debuggingFeaturesAllowed': True
}

androidmanagement.enterprises().policies().patch(
    name=policy_name,
    body=policy
).execute()

## Option 2: Set managed configurations through policy

In [ ]:
# Pull the managed configurations of the application to edit in the next code snippet below

application = androidmanagement.enterprises().applications().get(
  name= enterprise_name + '/applications/' + package_name
).execute()

prettyprint(application["managedProperties"])

Create your own UI and apply the settings via managedConfiguration in ApplicationPolicy.

In [ ]:
policy_name = enterprise_name + '/policies/managed-configurations'

policy = {
    'applications': [
        {
            'installType': 'FORCE_INSTALLED',
            'packageName': package_name,
            'managedConfiguration': {
                # Enter your configuration using this key/value pair template.
                'key1': 'value1',
                'key2': 'value2'
            }
        }
    ],
    'debuggingFeaturesAllowed': 'true'
}

# Write the new policy:
androidmanagement.enterprises().policies().patch(
    name=policy_name,
    body=policy
).execute()

# Retrieve feedback from the configured app

This section shows you how to [retrieve feedback from apps](https://developers.google.com/android/management/app-feedback) through the Android Management API. This communication channel allows IT admins to receive feedback about the status of the apps installed on the devices they manage.



Below is an example of how to create or update a `policy` for apps on a device with settings controlling the behavior of status reports. To enable reporting for a device, update the device policy and set StatusReportingSettings.applicationReportsEnabled to true. To enable device settings reporting set StatusReportingSettings.deviceSettingsEnabled to true.



In [ ]:
# Update a policy without replacing the existing policy.

policy_name = enterprise_name + '/policies/managed-configurations'

policy = {
    'statusReportingSettings' : {
        'applicationReportsEnabled': True
    }
}

androidmanagement.enterprises().policies().patch(
    name=policy_name,
    updateMask='statusReportingSettings.applicationReportsEnabled',
    body=policy
).execute()

In [ ]:
# List devices so you can get a device name to use to pull app feedback below.
androidmanagement.enterprises().devices().list(
    parent=enterprise_name
).execute()

To review a devices's latest report at any time, call [devices.get()](https://developers.google.com/android/management/reference/rest/v1/enterprises.devices/get) 

IT admins can use the data from keyed app states to retrieve information about device states.

In [ ]:
# Enter the device name
device_name = ''

In [ ]:
# Review a device’s latest report:
androidmanagement.enterprises().devices().get(
    name=device_name
).execute()

#   Set up [Pub/Sub notifications](https://developers.google.com/android/management/notifications)

As an alternative you can set up Pub/Sub notifications to receive alerts about newly enrolled devices, device reports, and recently issued commands. To set up Pub/Sub notifications, you need to enable the Pub/Sub API and create a topic. To receive messages published to a topic, create a subscription to that topic. After you create a subscription, you need to grant Android Device Policy permission to publish to your topic.

The set up link above will guide you on how to: 

1. Enable the Pub/Sub API for your project
2. Create a topic
3. Create a subscription
4. Grant Android Device Policy the right to publish to your topic
5. Update an enterprise to support notifications

Go to your [Cloud Console](https://console.cloud.google.com/cloud-resource-manager) to get your project ID.

In [ ]:
# Enter your Google Cloud Project ID below:
cloud_project_id = ''

In [ ]:
# Go to the Pub/Sub topics page in the Cloud Console to get the topic name you created.
# Enter the topic you created below:
my_topic = ''

In [ ]:

# Update the enterprise to support notifications:
androidmanagement.enterprises().patch(
    name=enterprise_name,
    body={
        "name" : enterprise_name,
        "enabledNotificationTypes" : "STATUS_REPORT",
        "pubsubTopic": "projects/" + cloud_project_id + "/topics/" + my_topic
    }
).execute()